In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import matplotlib.ticker as ticker
import geopandas
from shapely.geometry import shape
from PIL import Image
import glob
from IPython.display import HTML

# Load data from nytimes
# Has data for each county in each state
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', 
                 parse_dates=['date'])


# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

@np.vectorize
def state_abbrev(state):
    return us_state_abbrev[state]


In [ ]:
# Print the data
df.head()

In [ ]:
# Select South Carolina
state = df.copy(deep=True)
state["name"] = state.county + " ," + state_abbrev(state.state)
# Group by county for each date
state = state.pivot(index='date', columns='name', values='cases')
state = state.fillna(0.0)
state.tail(5)

In [ ]:
#plot = state.plot(figsize=(12,8), linewidth=5, cmap='PuBu')
#plot.legend(ncol=2, bbox_to_anchor=(1, 1), loc='upper left')
#plt.show()

In [ ]:
# Copy data
new_per_day = state.copy(deep=True)

# Get difference per day instead of total cases
for county in state.columns:
    new_per_day[county] = new_per_day[county].sum()
new_per_day = new_per_day.fillna(0.0)
new_per_day.tail(5) 


In [ ]:
# Get geometry data
# https://github.com/deldersveld/topojson
geodata = geopandas.read_file(f"https://github.com/deldersveld/topojson/blob/master/countries/united-states/us-albers-counties.json?raw=true")

geodata

In [ ]:
geodata['full_name'] = geodata.name + " ," + state_abbrev(geodata.state)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
geodata.plot(color="white", edgecolor='black', ax=ax)
max_count = np.max(state.max(axis=0))

geodata["color"] = np.zeros_like(geodata.full_name)

for c in state.iteritems():
    geodata.loc[geodata.full_name == c[0], 'color'] = c[1][-1]

geodata.plot(column='color', ax=ax, cmap='hot_r')
ax.set_title(f"Current Cases")
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)
plt.show()
plt.close()